# The Copper-to-Gold Ratio Using the OpenBB SDK

The copper-to-gold ratio is known as a leading economic indicator.  It is most commonly paired as a time series with the ten-year US Treasury yield. The notable events are the divergences in directional movement, signaling a fundamental regime change that will unfold over months and years.  Not something to go YOLO into, but a metric to shape a long-term view of global economic conditions.

The ratio is defined as dividing the spot price of one ounce of copper by an ounce gold.  How much copper is bought with one ouce of gold.  Sounds simple enough,  divide the price of copper by the price of gold, done.  The OpenBB SDK can make quick work out of this task, really quick.  Let's explore.

In [11]:
# Import the SDK and Pandas for some calculations.
from datetime import datetime
import pandas as pd
from openbb_terminal.sdk import openbb

from openbb_terminal import OpenBBFigure

The most accessible data is going to be the continuous front-month futures contracts for physical delivery, listed on the CME.  We'll create a Pandas Series for each asset, requesting all the daily historical data that is available.

In [12]:
data = pd.DataFrame(
    openbb.futures.historical(
        symbols = ["GC", "HG"],
        start_date="2000-01-01",
        end_date = str(datetime.now().date())
    )["Close"]
).rename(
    columns={"GC": "Gold", "HG": "Copper"}
)

Let's inspect the results.

In [13]:
data.head(2)

,Gold,Copper
Date,,
2000-08-30,273.899994,0.885
2000-08-31,278.299988,0.885


To get the copper-to-gold ratio, divide the two columns along each row.

In [14]:
data["Copper/Gold Ratio"] = data['Copper']/data['Gold']

data.tail(2)

,Gold,Copper,Copper/Gold Ratio
Date,,,
2023-05-24,1962.800049,3.5535,0.001810
2023-05-25,1943.099976,3.5805,0.001843


Because the numbers are so small, the ratio is often be presented as a % value.  0.2% is a popular way to display the value.  However, to plot it on the same y-axis as a Treasury yield, it needs to be multiplied by 1000.  Let's alter the block above to include this.

In [15]:
data["Copper/Gold Ratio"] = (data['Copper']/data['Gold']) * 1000

data.tail(2)

,Gold,Copper,Copper/Gold Ratio
Date,,,
2023-05-24,1962.800049,3.5535,1.810424
2023-05-25,1943.099976,3.5805,1.842674


Now let's add a column for the daily 10 Year US Treasury Yield.  This can be requested using the `fred` function within the `economy` module.  The first line in the block below requests the data, the second assigns it to a column in the target DataFrame.

In [16]:
us10year = openbb.economy.fred(['DGS10'])[0]
data['US 10-Year Constant Maturity'] = us10year["2000-08-30":]

data.head(2)

,Gold,Copper,Copper/Gold Ratio,US 10-Year Constant Maturity
Date,,,,
2000-08-30,273.899994,0.885,3.231106,5.81
2000-08-31,278.299988,0.885,3.180022,5.73


With all the data collected, let's draw the chart to visualize the relationship.

In [17]:
fig = OpenBBFigure()
fig.add_scatter(x=data.index, y=data["Copper/Gold Ratio"], name="Copper/Gold Ratio (x1000) %")
fig.add_scatter(x=data.index, y=data["US 10-Year Constant Maturity"], name="US 10-Year Constant Maturity %")
fig.update({'layout': 
    {'xaxis': {'title': 'Date'},
    'yaxis': {'title': '%'},
    'title': 'Copper/Gold Ratio vs. US 10-Year Constant Maturity',
    'title_y':0.90,
    'title_x':0.5,
}})
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.0
    )
)

There is another way to create the figure object for the chart.  Let's use the `plot` function from the `forecast` module to demonstrate.

In [18]:
fig = openbb.forecast.plot(data = data, columns = ["Copper/Gold Ratio","US 10-Year Constant Maturity"], external_axes = True)

fig.show()

Let's add in the title and also move the legend to the right side of the chart.

In [19]:
fig.update({'layout': 
    {'xaxis': {'title': 'Date'},
    'yaxis': {'title': '%'},
    'title': 'Copper/Gold Ratio vs. US 10-Year Constant Maturity',
    'title_y':0.98,
    'title_x':0.5,
}})
fig.update_layout(
        legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.0
    )   
)

What we have currently is the price relationship between one Troy ounce of gold and one pound of copper.  As we described the copper-to-gold ratio as the price-per-ounce of each, some adjustments are required to be true to the definition.

- 1 ounce = 0.911458 Troy ounces
- 1 pound = 16 ounces
  
To adjust the gold price as USD/ounce, multiply each row by 0.911458.  To adjust the copper price, divide each row by 16.

In [25]:
data['Copper/Gold Ratio per Ounce (x1000) %'] = (
    ((data['Copper']/16)/(data['Gold']*0.911458))*1000
)

data.tail(2)

,Gold,Copper,Copper/Gold Ratio,US 10-Year Constant Maturity,Copper/Gold Ratio per Ounce (x1000) %
Date,,,,,
2023-05-24,1962.800049,3.5535,1.810424,3.73,0.124143
2023-05-25,1943.099976,3.5805,1.842674,NaN,0.126355


The conversion to a per-ounce relationship has decreased the number where it wouldn't compare well on the same y-axis. Let's import the plotting function from the `/economy` menu in the OpenBB Terminal.

In [21]:
from openbb_terminal.economy.plot_view import show_plot 

In [22]:
show_plot?

Signature:
show_plot(
    dataset_yaxis_1,
    dataset_yaxis_2,
    export: str = '',
    sheet_name: Optional[str] = '',
    external_axes: bool = False,
)
Docstring:
The ability to plot any data coming from EconDB, FRED or Yahoo Finance.

Parameters
----------
dataset_yaxis_1: pd.DataFrame
    Data you wish to plot on the first y-axis.
dataset_yaxis_2 : pd.DataFrame
    Data you wish to plot on the second y-axis.
external_axes : bool, optional
    Whether to return the figure object or not, by default False

Returns
-------
Plots the data.
File:      ~/GitHub/OpenBBTerminal/openbb_terminal/economy/plot_view.py
Type:      function

Each y-axis requires its own DataFrame, so let's make one for each next.

In [23]:
y1 = pd.DataFrame(data["Copper/Gold Ratio per Ounce (x1000) %"])
y2 = pd.DataFrame(data["US 10-Year Constant Maturity"])
fig = show_plot(y1, y2, external_axes = True)

In [24]:
fig.update({'layout': 
    {'xaxis': {'title': 'Date'},
    'yaxis': {'title': 'Copper/Gold Ratio'},
    'yaxis2': {'title': '10Y Yield'},
    'title': 'Copper/Gold Ratio vs. US 10-Year Constant Maturity',
    'title_y':0.98,
    'title_x':0.5,
}})
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=0.9
    )
)

There you have it, folks!  The OpenBB SDK provides endless possibilities for creating unique indicators and analysis with the wide variety of data available at your fingertips.  We love seeing the creations of users, so be sure to tag us on social media and show off your work.